# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Fall 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [9]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter
  
print(kb.kb_triples[89])
from collections import Counter

simple_bag_of_words_featurizer(kb.kb_triples[89], corpus, Counter())

class foo:
  c = 7
  def __init__(self):
    self.a = 6
    b = 9
k = foo()
print(k.__dict__)

KBTriple(rel='contains', sbj='Derry', obj='Millennium_Forum')
{'a': 6}


In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [12]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.853      0.391      0.690        340       5716
author                    0.780      0.536      0.715        509       5885
capital                   0.724      0.221      0.498         95       5471
contains                  0.790      0.608      0.745       3904       9280
film_performance          0.777      0.560      0.721        766       6142
founders                  0.777      0.384      0.645        380       5756
genre                     0.609      0.165      0.395        170       5546
has_sibling               0.854      0.246      0.572        499       5875
has_spouse                0.845      0.338      0.650        594       5970
is_a                      0.688      0.195      0.457        497       5873
nationality               0.670      0.203      0.459        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.538 Córdoba
     2.428 Taluks
     2.414 Valais
     ..... .....
    -1.070 capital
    -1.158 Afghanistan
    -1.349 America

Highest and lowest feature weights for relation author:

     2.527 books
     2.495 author
     2.434 wrote
     ..... .....
    -2.283 directed
    -2.748 Alice
    -6.999 1865

Highest and lowest feature weights for relation capital:

     3.428 capital
     1.821 km
     1.770 city
     ..... .....
    -1.088 or
    -1.194 and
    -1.267 also

Highest and lowest feature weights for relation contains:

     2.192 bordered
     2.074 third-largest
     2.073 southwestern
     ..... .....
    -2.691 Mile
    -3.440 Midlands
    -3.630 Ceylon

Highest and lowest feature weights for relation film_performance:

     4.262 starring
     3.781 co-starring
     3.298 alongside
     ..... .....
    -2.093 historical
    -2.097 Iruvar
    -3.987 double

Highest and lowest feature weights for relation fou

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [14]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [15]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [16]:
import itertools
#print(dict(itertools.islice(glove_lookup.items(), 8)))

In [17]:
def glove_middle_featurizer(kbt, corpus, np_func=np.mean):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [18]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.340      0.050      0.157        340       5716
author                    0.883      0.356      0.681        509       5885
capital                   0.462      0.063      0.204         95       5471
contains                  0.578      0.512      0.564       3904       9280
film_performance          0.743      0.268      0.548        766       6142
founders                  0.717      0.113      0.347        380       5756
genre                     0.625      0.059      0.214        170       5546
has_sibling               0.788      0.082      0.290        499       5875
has_spouse                0.745      0.192      0.473        594       5970
is_a                      0.600      0.042      0.165        497       5873
nationality               0.729      0.143      0.400        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [19]:
def run_svm_model_factory():

  ##### YOUR CODE HERE
  from sklearn.svm import SVC
  
  svc_model_factory = lambda: SVC(kernel='linear')
  glove_svc_results = rel_ext.experiment(
    splits,
    #AH TMP
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],
    model_factory=svc_model_factory,
    vectorize=False, # Crucial for this featurizer!
    verbose=True)
  
  return glove_svc_results

#run_svm_model_factory()

In [20]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."

In [21]:
#if 'IS_GRADESCOPE_ENV' not in os.environ:
   #test_run_svm_model_factory(run_svm_model_factory)

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [22]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter):
    # Append these to the end of the keys you add/access in
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
directional_unigram_results = rel_ext.experiment(
    splits,
    featurizers=[directional_bag_of_words_featurizer])


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.857      0.424      0.711        340       5716
author                    0.835      0.587      0.770        509       5885
capital                   0.735      0.263      0.541         95       5471
contains                  0.821      0.654      0.781       3904       9280
film_performance          0.842      0.662      0.799        766       6142
founders                  0.814      0.403      0.676        380       5756
genre                     0.710      0.259      0.526        170       5546
has_sibling               0.853      0.244      0.570        499       5875
has_spouse                0.863      0.360      0.675        594       5970
is_a                      0.747      0.237      0.523        497       5873
nationality               0.673      0.219      0.476        301       5677
parents     

In [23]:
print(f"Number of features names that the vectorizer has: {len(directional_unigram_results['vectorizer'].get_feature_names())}")

Number of features names that the vectorizer has: 40576


In [24]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [25]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [26]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_tag_bigrams(ex.middle_POS):
            feature_counter[word] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"

    ##### YOUR CODE HERE
    tags = [start_symbol] + get_tags(s) + [end_symbol]
    tags = [tags[i-1] + ' ' + tags[i] for i in range(1, len(tags))]
    return tags


def get_tags(s):
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
part_of_speech_results = rel_ext.experiment(
    splits,
    featurizers=[middle_bigram_pos_tag_featurizer])


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.808      0.347      0.639        340       5716
author                    0.714      0.338      0.584        509       5885
capital                   0.640      0.168      0.410         95       5471
contains                  0.774      0.592      0.729       3904       9280
film_performance          0.715      0.448      0.638        766       6142
founders                  0.520      0.174      0.372        380       5756
genre                     0.612      0.176      0.410        170       5546
has_sibling               0.624      0.166      0.403        499       5875
has_spouse                0.764      0.278      0.566        594       5970
is_a                      0.557      0.157      0.369        497       5873
nationality               0.389      0.070      0.203        301       5677
parents     

In [27]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [28]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [29]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):

    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in get_synsets(ex.middle_POS):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in get_synsets(ex.middle_POS):
            feature_counter[word] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets
    associated with elements of `s`.
    """
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    synsets = [wn.synsets(w.lower(), convert_tag(t)) for w, t in wt]

    str_synsets = [str(s) for syn in synsets for s in syn]
    return str_synsets



def convert_tag(t):
    """Converts tags so that they can be used by WordNet:

    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
synset_results = rel_ext.experiment(
    splits,
    featurizers=[synset_featurizer], verbose=True)

/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.725      0.326      0.583        340       5716
author                    0.772      0.446      0.674        509       5885
capital                   0.719      0.242      0.516         95       5471
contains                  0.777      0.596      0.733       3904       9280
film_performance          0.773      0.552      0.716        766       6142
founders                  0.735      0.366      0.612        380       5756
genre                     0.493      0.212      0.390        170       5546
has_sibling               0.787      0.222      0.522        499       5875
has_spouse                0.833      0.310      0.622        594       5970
is_a                      0.597      0.229      0.452        497       5873
nationality               0.456      0.136      0.310        301       5677
parents     

In [30]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [31]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best score your system got during development, just to help us understand how systems performed overall.

In [32]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# IMPORT ANY MODULES BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: 0.572
# My system creates a very simple bag of words feature dictionary using the context of the full sentences where the 
# two entities are jointly mentioned. As a reweighting mechanism it uses a scaled weight that is proportional to the distance
# between the word and its closest entity mention. This is only applicable for the left & right half of the context. For the middle
# it simply uses a scaling of 1, in other words all middle words have the full and equal weight. Additionally it also adds the two 
# entity mentions themselves in the feature dictionary. Finally this scaled context window featurizer is uses in both forward & reverse
# for the entity mentions. I played around with a few variants of scaling etc, but this configuration worked best.
# I briefly played around with using Adaboost classifier but that doesn't seem to work as well so I just used 
# the default LogisticRegression classifier.

if 'IS_GRADESCOPE_ENV' not in os.environ:
  
  print('start')
  
  def scaled_featurizer(kbt, corpus, feature_counter):
    
    def add_scaled_words(ex, feature_counter):
      left_tokens = ex.left.split()
      for idx, word in enumerate(left_tokens):
        feature_counter[word] += (idx/len(left_tokens))
      mid_tokens = ex.middle.split()
      for idx, word in enumerate(mid_tokens):
        feature_counter[word] += 1
      right_tokens = ex.right.split()        
      for idx, word in enumerate(right_tokens):
        feature_counter[word] += (len(right_tokens) - idx)/len(right_tokens)
      for mention in ex.mention_1.split():
        feature_counter[mention] += 1
      for mention in ex.mention_2.split():
        feature_counter[mention] += 1
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
      add_scaled_words(ex, feature_counter)

    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
      add_scaled_words(ex, feature_counter)

    return feature_counter  

  # Original System # 1
  orig_sys_1_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[scaled_featurizer],
    verbose=True)
  
# STOP COMMENT: Please do not remove this comment.

start
relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.890      0.500      0.770        340       5716
author                    0.774      0.517      0.704        509       5885
capital                   0.513      0.211      0.398         95       5471
contains                  0.780      0.606      0.738       3904       9280
film_performance          0.791      0.648      0.757        766       6142
founders                  0.655      0.334      0.549        380       5756
genre                     0.716      0.312      0.569        170       5546
has_sibling               0.881      0.669      0.829        499       5875
has_spouse                0.724      0.327      0.582        594       5970
is_a                      0.725      0.531      0.676        497       5873
nationality               0.556      0.199      0.409        301       5677
parent

In [33]:
  # Original system # 2
  # This system was a different one I built that did not work that well.
  # The idea here was to simply take trigrams of the full sentence with the 2 mentions,
  # look up the GloVe vectors for each word and average them over the trigram
  
  def get_trigrams(s):
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    s = [start_symbol] + s.split() + [end_symbol]
    trigrams = [(s[i-2], s[i-1], s[i]) for i in range(2, len(s))]
    return trigrams 
  
  GLOVE_HOME = os.path.join('data', 'glove.6B')
  glove_lookup = utils.glove2dict(
   os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

  def trigram_glove_featurizer(kbt, corpus):
    reps = []
    dim = len(next(iter(glove_lookup.values())))
    
    def featurize(ex, reps):
      sent = ex.left + ex.mention_1 + ex.middle + ex.mention_2 + ex.right
      trigrams = get_trigrams(sent)
      for t in trigrams:
        tws = [np.zeros(dim), np.zeros(dim), np.zeros(dim)]
        for i, word in enumerate(t):
          tw = glove_lookup.get(word)
          if tw is not None:
            tws[i] = tw          
        reps.append(np.sum(tws, axis=0))
      return reps
    
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
      reps += featurize(ex, reps)
      
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
      reps += featurize(ex, reps)
        
    # A random representation of the right dimensionality if 
    # nothing was found in GloVe or the corpus.
    if len(reps) == 0:
        return utils.randvec(n=dim)
    else:
        return np.mean(reps, axis=0) 

#   glove_results_1 = rel_ext.experiment(
#     splits,
#     train_split='train',
#     test_split='dev',
#     featurizers=[glove_middle_featurizer],
#     vectorize=False, # Crucial for this featurizer!
#     verbose=True)


## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [36]:
# Enter your bake-off assessment code in this cell.
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    rel_ext_data_home_test = os.path.join(
      rel_ext_data_home, 'bakeoff-rel_ext-test-data')
    orig_sys_bakeoff_results = rel_ext.experiment(
      splits,
      train_split='all',
      test_split='dev',
      featurizers=[scaled_featurizer],
      verbose=True)
    
    print("-----------------------")
    rel_ext.bake_off_experiment(orig_sys_bakeoff_results, rel_ext_data_home_test)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.877      0.524      0.773        340       5716
author                    0.862      0.823      0.854        509       5885
capital                   0.733      0.663      0.718         95       5471
contains                  0.828      0.867      0.836       3904       9280
film_performance          0.848      0.958      0.868        766       6142
founders                  0.848      0.824      0.843        380       5756
genre                     0.880      0.947      0.892        170       5546
has_sibling               0.903      0.687      0.849        499       5875
has_spouse                0.871      0.672      0.822        594       5970
is_a                      0.822      0.789      0.815        497       5873
nationality               0.768      0.595      0.726        301       5677
parents     

In [35]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported
# by the code above. Please enter only a number between
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass
    # Please enter your score in the scope of the above conditional.
    0.587
